In [1]:
"""
Geotiff Rasterization Script

This Python script demonstrates how to rasterize vector data from a geodatabase onto a geotiff image for multiple years. The script uses the GDAL library for geospatial data processing and the OGR library for reading vector data.

The purpose of the script is to convert vector data from the geodatabase into raster format (geotiff) based on certain criteria. The input vector data is filtered using specific attribute queries for each year from a defined range (1982 to 1983 in this example).

The script's functionality includes the following steps:
1. Define the file paths for the geodatabase and the reference geotiff file (1982).
2. Open the reference geotiff file to get its size, resolution, geotransform, and projection information.
3. Create a new folder named "rasterized" within the GLASS-GLC directory to store the rasterized geotiff files for each year.
4. Open the geodatabase and obtain the first layer.
5. Loop through each year and apply attribute filtering to the vector data based on the 'STATUS_YR' attribute, generating a new query for each year.
6. Create a new raster dataset for each year's rasterized geotiff using the GDAL driver.
7. Rasterize the filtered vector data onto the new raster dataset, setting a specific burn value for the rasterized area.
8. Save the rasterized geotiff for each year in the "rasterized" folder.
9. Clean up resources after rasterization is complete.

The script uses GDAL's gdal.RasterizeLayer function to rasterize the vector data onto the geotiff dataset.

Note:
- Before running the script, ensure that the required GDAL and OGR libraries are installed.
- The code provided is a simplified example, and additional error handling and parameterizations may be necessary for more complex use cases.

Author: Matthew Braaksma
Date: 08-03-2023
"""

from osgeo import ogr, gdal
import os

# Geodatabase file path
geodatabase = 'WDPA_Jan2023_Public.gdb'
geodatabase_path = '../../data/WDPA/raw/' + geodatabase

# Geotiff file path
# Use single year as reference
geotiff_path = "../../data/GLASS-GLC/raw/GLASS-GLC_7classes_1982.tif"

# Subset parameters
cr_xoff = 2095
cr_yoff = 1640
cr_xsize = 80
cr_ysize = 70

# List of years from 1982 to 2015
years = list(range(1982, 1985))

# Open the geotiff file to get its size and resolution
geotiff_dataset = gdal.Open(geotiff_path, gdal.GA_ReadOnly)
if geotiff_dataset is None:
    print("Error: Could not open the geotiff file.")
    exit(1)

# Get the geotransform and projection of the geotiff
geotransform = geotiff_dataset.GetGeoTransform()
projection = geotiff_dataset.GetProjection()

# Get the size (width and height) of the geotiff
width = geotiff_dataset.RasterXSize
height = geotiff_dataset.RasterYSize

# Create the output folder for rasterized geotiffs
output_folder = os.path.join("../../data/GLASS-GLC", "rasterized")
os.makedirs(output_folder, exist_ok=True)

# Open the geodatabase
geodatabase_dataset = ogr.Open(geodatabase_path, 0)  # 0 for read-only mode
if geodatabase_dataset is None:
    print("Error: Could not open the geodatabase.")
    exit(1)

# Get the first layer in the geodatabase
layer = geodatabase_dataset.GetLayerByIndex(0)

# Loop through each year
for year in years:
    # Create a query for the current year based on 'STATUS_YR' attribute
    query = f"ISO3 = 'CRI' AND MARINE = 0 AND STATUS_YR <= {year}"
    layer.SetAttributeFilter(query)

    # Create a new raster dataset for rasterization in memory
    driver = gdal.GetDriverByName("GTiff")
    rasterized_path = f"../../data/GLASS-GLC/rasterized/rasterized_{year}.tif"
    rasterized_dataset = driver.Create(rasterized_path, cr_xsize, cr_ysize, 1, gdal.GDT_Byte)
    rasterized_dataset.SetGeoTransform((geotransform[0] + cr_xoff * geotransform[1],
                                        geotransform[1],
                                        geotransform[2],
                                        geotransform[3] + cr_yoff * geotransform[5],
                                        geotransform[4],
                                        geotransform[5]))
    rasterized_dataset.SetProjection(projection)

    # Set the burn value (value to be assigned to the rasterized area)
    burn_value = 1

    # Rasterize the layer
    gdal.RasterizeLayer(rasterized_dataset, [1], layer, burn_values=[burn_value])

    # Clean up
    rasterized_dataset = None

# Clean up
geotiff_dataset = None
geodatabase_dataset = None

print("Rasterization completed for each year.")

Rasterization completed for each year.
